In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
df = pd.read_csv("../csv/vehicles_us.csv")

In [ ]:
def cleaning(data):
    """
    Limpieza básica del dataset de vehículos.

    Pasos:
    1. Rellenar valores faltantes (fillna) según el tipo de columna.
    2. Estandarizar formatos y tipos de datos.
    3. Aplicar correcciones específicas por columna.
    """

    # ----------------------------------------------------------------------#
    # 1️⃣ RELLENAR VALORES FALTANTES
    # ----------------------------------------------------------------------#
    # Sustituye 1 → 'Yes' y NaN → 'No' en columna 'is_4wd'
    data['is_4wd'] = data['is_4wd'].replace({1: 'Yes', np.nan: 'No'})

    # Completa odometer, cylinders y model_year con 0 si están vacíos
    data['odometer'] = data['odometer'].fillna(0)
    data['cylinders'] = data['cylinders'].fillna(0)
    data['model_year'] = data['model_year'].fillna(0)

    # Completa paint_color con un valor genérico
    data['paint_color'] = data['paint_color'].fillna('random')

    # Rellena 'cylinders' según el modo de cada modelo (si falta y existe)
    data['cylinders'] = data.groupby("model")['cylinders'].transform(
        lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan)
    )

    # ----------------------------------------------------------------------#
    # 2️⃣ CAMBIOS DE TIPO DE DATO
    # ----------------------------------------------------------------------#
    # Convierte fechas y números a su tipo correcto
    data['date_posted'] = pd.to_datetime(data['date_posted'])
    data['odometer'] = data['odometer'].astype(int)
    data['cylinders'] = data['cylinders'].astype(int)
    data['model_year'] = data['model_year'].astype(int)

    # ----------------------------------------------------------------------#
    # 3️⃣ RETORNAR DATA LIMPIA
    # ----------------------------------------------------------------------#
    return data


df_clean = cleaning(df)

## Grafica uno

In [ ]:
fig = px.scatter(
    df,
    x="odometer",             # eje X: kilometraje
    y="price",                # eje Y: precio
    color="condition",        # color por condición del vehículo
    size="model_year",        # tamaño del punto según el año
    hover_data=["model", "type"],  # información al pasar el cursor
    title="Relación entre kilometraje, precio y condición del vehículo",
    labels={
        "odometer": "Kilometraje recorrido",
        "price": "Precio (USD)",
        "condition": "Condición del vehículo",
        "model_year": "Año del modelo"
    },
    opacity=0.6
)

fig.show()

## Grafica dos

In [ ]:
cond_mean = df.groupby("condition", as_index=False)["price"].mean()

fig2 = px.bar(
    cond_mean,
    x="condition",
    y="price",
    title="Precio promedio según condición del vehículo",
    labels={"condition": "Condición", "price": "Precio promedio (USD)"},
)
fig2.update_traces(marker_color="skyblue")
fig2.show()

## Grafica tres

In [ ]:
type_mean = df.groupby("type", as_index=False)["price"].mean()

fig4 = px.bar(
    type_mean,
    x="price",
    y="type",
    orientation="h",
    title="Precio promedio por tipo de vehículo",
    labels={"type": "Tipo de vehículo", "price": "Precio promedio (USD)"},
)
fig4.update_traces(marker_color="coral")
fig4.show()